In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
# -*- coding: utf-8 -*-
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
warnings.filterwarnings("ignore")  # 选择过滤警告
import scipy.io as scio
import scipy
import statsmodels.tsa.api as smt
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import coint
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import warnings


import datetime
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels import regression
from six import StringIO
#导入pca
from sklearn.decomposition import PCA
from sklearn.svm import SVC,SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

import seaborn as sns
import os

pd.set_option('display.max_rows',500)
pd.set_option('display.min_rows',500)

In [ ]:
# 读取数据
data=pd.read_excel('data.xlsx',index_col='dates')
data['自由流通市值']=data['自由流通股本']*data['收盘价']
price=data['收盘价']
op=data['开盘价']
dates=data.index
dates=dates[(dates>'20100101') & (dates<'20210101')] #回测2010-2020数据
factors=data[['cpi_month', 'pmi', 'ind_growth_yoy', 'fixed_assets_investment',
       'consistency_idx','early_warning_idx',
       'foreign','m2', 'm1', 'retail_sin',
        'gdp_sin','commodity_house_idx',
       '十年国债', '成交额', '换手率', '自由流通市值', 'PE(TTM)', '股息率', 'PB(LR)',
       '一致预测净利润同比', '一致预测每股收益(FY1)', 'macd', 'kdj',
       'rsi', 'atr', 'ROE(平均)', '归母净利润同比增长率', 'EPS', '营业收入同比增长率']]
# 预测周期
m=19
# 训练周期
k=235
#tag值为一个月后的涨幅
price_ch=price.shift(-m)/price-1

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(price)
plt.legend(['万得全A指数收盘价'])

In [ ]:
len(['cpi_month', 'pmi', 'ind_growth_yoy', 'fixed_assets_investment',
       'consistency_idx','early_warning_idx',
       'foreign','m2', 'm1', 'retail_sin',
        'gdp_sin','commodity_house_idx',
       '十年国债', '成交额', '换手率', '自由流通市值', 'PE(TTM)', '股息率', 'PB(LR)',
       '一致预测净利润同比', '一致预测每股收益(FY1)', 'macd', 'kdj',
       'rsi', 'atr', 'ROE(平均)', '归母净利润同比增长率', 'EPS', '营业收入同比增长率'])

In [ ]:
# 小波变换处理收盘价
import pywt # 小波分析
import scipy.fftpack as fftpack # 希尔伯特变换

In [ ]:
#对指数时间序列进行小波分解，小波函数选择为db8，分解层个数为4个，即一个主要趋势，和四个细节成分
#四个参数分别 对应 输入的信号 ，小波名称， 信号扩展模式， 分解阶次  
res_4 = pywt.wavedec(price_ch.dropna(),wavelet='db8',mode='smooth',level=4) 
col_name = ['主趋势'] + [f'细节{i}' for i in range(1,5)]
res4_df = pd.DataFrame(res_4,index=col_name).T
res4_df.head()

In [ ]:
res4_df.iloc[:, 1:].plot(subplots=True, layout=(2, 2), figsize=(18, 8), sharex=False)
plt.show()
#从上图可以看到，相对而言，细节3和4更像是高频的噪声信号，故对这两个细节进行阈值降噪处理。
#这里涉及到阈值选择的问题。 通常来说，阈值选择有：1、无偏风险估计阈值(rigrsure) 2、固定阈值 
#3、启发式阈值 4、极大极小阈值 这些方法，在这里选择rigsure方法来确定阈值

In [ ]:
# 信号去噪
class DenoisingThreshold(object):
    '''
    获取小波去噪的阈值
    1. CalSqtwolog 固定阈值准则(sqtwolog)
    2. CalRigrsure 无偏风险估计准则（rigrsure）
    3. CalMinmaxi 极小极大准则（ minimaxi）
    4. CalHeursure
    参考：https://wenku.baidu.com/view/63d62a818762caaedd33d463.html
    对股票价格等数据而言，其信号频率较少地与噪声重叠因此可以选用sqtwolog和heursure准则，使去噪效果更明显。 
    但对收益率这样的高频数据，尽量采用保守的 rigrsure 或 minimaxi 准则来确定阈值，以保留较多的信号。
    '''
    def __init__(self,signal: np.array):
        self.signal = signal
        self.N = len(signal)
    # 固定阈值准则(sqtwolog)
    @property
    def CalSqtwolog(self) -> float:
        return np.sqrt(2 * np.log(self.N))
   
    # 无偏风险估计准则（rigrsure）
    @property
    def CalRigrsure(self)->float:
        N = self.N
        signal = np.abs(self.signal)
        signal = np.sort(signal)
        signal = np.power(signal, 2)
        risk_j = np.zeros(N)
        for j in range(N):
            if j == 0:
                risk_j[j] = 1 + signal[N - 1]
            else:
                risk_j[j] = (N - 2 * j + (N - j) *
                             (signal[N - j]) + np.sum(signal[:j])) / N
        k = risk_j.argmin()
        return np.sqrt(signal[k])
    # 极小极大准则（ minimaxi）
    @property
    def CalMinmaxi(self)->float:
        if self.N > 32:
            # N>32 可以使用minmaxi阈值 反之则为0
            return 0.3936 + 0.1829 * (np.log(self.N) / np.log(2))
        else:
            return 0
    @property
    def GetCrit(self)->float:
        return np.sqrt(np.power(np.log(self.N) / np.log(2), 3) * 1 / self.N)
    @property
    def GetEta(self)->float:
        return (np.sum(np.abs(self.signal)**2) - self.N) / self.N
    #混合准则（heursure）
    @property
    def CalHeursure(self):
        if self.GetCrit > self.GetEta:
            #print('推荐使用sqtwolog阈值')
            return self.CalSqtwolog
        else:
            #print('推荐使用 Min(sqtwolog阈值,rigrsure阈值)')
            return min(self.CalRigrsure,self.CalSqtwolog)

In [ ]:
#对于高频的信号进行阈值处理
for j in [3,4]:     
    signal = res_4[j]
    threshold = DenoisingThreshold(signal).CalRigrsure 
    res_4[j] = pywt.threshold(signal, threshold, 'soft')
rec_hs = pywt.waverec(res_4, 'db8')

In [ ]:
#得到小波处理后的股价
try:
    price_ch_new=pd.Series(rec_hs,index=price_ch.dropna().index)
except:
    price_ch_new=pd.Series(rec_hs[1:],index=price_ch.dropna().index)
price_ch_new.name='收盘价'
plt.figure(figsize=(15,4))
plt.plot(price_ch)
plt.plot(price_ch_new)
plt.legend(['原收益率曲线','小波降噪后曲线'])

In [ ]:
#是否使用小波降噪
price_ch=price_ch_new

In [ ]:
# 因子MA处理
factors['cpi_month']=factors['cpi_month'].rolling(120).mean()
factors['consistency_idx']=factors['consistency_idx'].rolling(60).mean()
factors['early_warning_idx']=factors['early_warning_idx'].rolling(60).mean()
factors['retail_sin']=factors['retail_sin'].rolling(60).mean()
factors['m1']=factors['m1'].rolling(60).mean()
factors['m2']=factors['m1'].rolling(60).mean()
factors['gdp_sin']=factors['gdp_sin'].rolling(120).mean()
factors['十年国债']=factors['十年国债'].rolling(60).mean()
factors['自由流通市值']=factors['自由流通市值']
factors['PE(TTM)']=factors['PE(TTM)'].rolling(20).mean()
factors['股息率']=factors['股息率'].rolling(60).mean()
factors['PB(LR)']=factors['PB(LR)'].rolling(120).mean()
factors['一致预测净利润同比']=factors['一致预测净利润同比'].rolling(60).mean()
factors['一致预测每股收益(FY1)']=factors['一致预测每股收益(FY1)'].rolling(60).mean()
factors['营业收入同比增长率']=factors['营业收入同比增长率'].rolling(60).mean()

In [ ]:
#去极值标准化
def mad(factor):
    me=np.median(factor)
    mad=np.median(abs(factor-me))
    up=me+1.4826*mad*3
    down=me-1.4826*mad*3
    factor=np.where(factor>up,up,factor)
    factor=np.where(factor<down,down,factor)
    return factor
def stand(factor):
    mean=factor.mean()
    std=factor.std()
    return (factor-mean)/std
factors=factors.apply(lambda x: mad(x),axis=0)
factors=factors.apply(lambda x: stand(x),axis=0)

In [ ]:
# 股价涨跌幅平稳性检验
plt.figure(figsize=(15,4))
plt.title('平稳性检验')
plt.plot(price_ch)
plt.show()
smt.adfuller(price_ch.dropna())

In [ ]:
#根据corr结果选择每个因子的滞后期
# factors['cpi_month']=factors['cpi_month'].shift(40)
# factors['fixed_assets_investment']=factors['fixed_assets_investment'].shift(120)
# factors['early_warning_idx']=factors['early_warning_idx'].shift(60)
factors['retail_sin']=factors['retail_sin'].shift(40)
factors['十年国债']=factors['十年国债'].shift(20)
factors['成交额']=factors['成交额'].shift(20)
factors['换手率']=factors['换手率'].shift(20)
factors['ind_growth_yoy']=factors['ind_growth_yoy'].shift(20)
factors['自由流通市值']=factors['自由流通市值'].shift(20)
factors['一致预测每股收益(FY1)']=factors['一致预测每股收益(FY1)'].shift(60)
factors['macd']=factors['macd'].shift(20)
factors['kdj']=factors['kdj'].shift(20)
factors['rsi']=factors['rsi'].shift(20)
factors['ROE(平均)']=factors['ROE(平均)'].shift(40)
factors['gdp_sin']=factors['gdp_sin'].shift(60)
factors['EPS']=factors['EPS'].shift(20)
factors['归母净利润同比增长率']=factors['归母净利润同比增长率'].shift(60)

In [ ]:
factors=factors[['cpi_month','ind_growth_yoy', 'fixed_assets_investment','retail_sin','consistency_idx','early_warning_idx','commodity_house_idx',
       'foreign','成交额', '换手率', '自由流通市值','十年国债', 'gdp_sin','PE(TTM)', '股息率',
                 '一致预测净利润同比','一致预测每股收益(FY1)','ROE(平均)', '归母净利润同比增长率','EPS',
                 '营业收入同比增长率','PB(LR)','macd', 'kdj','rsi']]